In [1]:
import math
import pandas as pd
import numpy as np
from collections import deque

# Contains the information of the node and another nodes of the Decision Tree.
class Node:
    def __init__(self):
        # value: attribute to make the split and branches.
        self.value = None
        # next: next node
        self.next = None
        # childs: branches coming off the decision nodes
        self.childs = None

# Decision Tree using ID3 algorithm.
class DecisionTree:
    def __init__(self, X, attribute_names, labels):
        # predictors
        self.X = X
        # names of the attributes
        self.attribute_names = attribute_names
        # categories
        self.labels = labels
        # list of unique categories
        self.labelCategories = list(set(labels))
        # nodes
        self.node = None

    # Calculates the entropy.
    # Parameters: x_ids - list containing the instances' IDs
    # Returns: entropy - float 
    def get_entropy(self, x_ids):
        # sorted labels by instance id
        labels = [self.labels[i] for i in x_ids]
        # count number of instances of each category
        label_count = [labels.count(x) for x in self.labelCategories]
        # calculate the entropy for each category and sum them
        entropy = sum([-count / len(x_ids) * math.log(count / len(x_ids), 2)
                    if count else 0
                    for count in label_count
                  ])
        return entropy

    # Calculates the information gain for a given attribute based on its entropy and the total entropy of the system.
    # Parameters:  x_ids - list containing the instances' IDs, attribute_id - int of attribute ID
    # Returns: info_gain - float representing the information gain for a given attribute.
    def get_information_gain(self, x_ids, attribute_id):
        # calculate total entropy
        info_gain = self.get_entropy(x_ids)
        # store in a list all the values of the chosen attribute
        x_attributes = [self.X[x][attribute_id] for x in x_ids]
        # get unique values
        attribute_vals = list(set(x_attributes))
        # get frequency of each value
        attribute_v_count = [x_attributes.count(x) for x in attribute_vals]
        # get the attribute values ids
        attribute_v_id = [
            [x_ids[i]
            for i, x in enumerate(x_attributes)
            if x == y]
            for y in attribute_vals
        ]
        # compute the information gain with the chosen attribute
        info_gain_attribute = sum([v_counts / len(x_ids) * self.get_entropy(v_ids)
                            for v_counts, v_ids in zip(attribute_v_count, attribute_v_id)])
                            # here, zip joins together v_count & v_id
        info_gain = info_gain - info_gain_attribute

        return info_gain
    
    # Finds the attribute that maximizes the information gain.
    # Parameters: x_ids - list containing the samples IDs, attribute_ids - list containing the attribute IDs
    # Returns: string and int, attribute and attribute id of the attribute that maximizes the information gain
    def get_attribute_max_information_gain(self, x_ids, attribute_ids):        
        # get the entropy for each attribute
        attributes_entropy = [self.get_information_gain(x_ids, attribute_id) for attribute_id in attribute_ids]
        # find the attribute that maximizes the information gain
        max_id = attribute_ids[attributes_entropy.index(max(attributes_entropy))]

        return self.attribute_names[max_id], max_id

    # Initializes ID3 algorithm to build a Decision Tree
    # Returns: None
    def id3(self):
        # assign an unique number to each instance
        x_ids = [x for x in range(len(self.X))]
        # assign an unique number to each feature
        attribute_ids = [x for x in range(len(self.attribute_names))]
        # define node variable - instance of the class Node
        self.node = self.id3_recv(x_ids, attribute_ids, self.node)

    # ID3 algorithm. It is called recursively until some criteria is met.
    # Parameters: x_ids - list containing the samples IDs
    #             attribute_ids: list containing the attribute IDs
    #             node: object instance of the class Node
    # Returns: An instance of the class Node containing all the information of the nodes in the Decision Tree
    def id3_recv(self, x_ids, attribute_ids, node):
        if not node:
            node = Node()  # initialize nodes
        # sorted labels by instance id
        labels_in_attributes = [self.labels[x] for x in x_ids]
        # if all the examples have the same class (pure node), return node
        if len(set(labels_in_attributes)) == 1:
            node.value = self.labels[x_ids[0]]
            return node
        # if there are not more attributes to compute, return node with the most probable class
        if len(attribute_ids) == 0:
            node.value = max(set(labels_in_attributes), key=labels_in_attributes.count) # compute mode
            return node
        # else, choose the attribute that maximizes the information gain
        best_attribute_name, best_attribute_id = self.get_attribute_max_information_gain(x_ids, attribute_ids)
        node.value = best_attribute_name
        node.childs = []
        # value of the chosen attribute for each instance
        attribute_values = list(set([self.X[x][best_attribute_id] for x in x_ids]))
        # loop through all the attribute_values
        for value in attribute_values:
            child = Node()
            child.value = value  # add a branch from the node to each attribute value in our attribute
            node.childs.append(child)  # append new child node to current node
            child_x_ids = [x for x in x_ids if self.X[x][best_attribute_id] == value]
            if not child_x_ids:
                child.next = max(set(labels_in_attributes), key=labels_in_attributes.count)
            else:
                if attribute_ids and best_attribute_id in attribute_ids:
                    to_remove = attribute_ids.index(best_attribute_id)
                    attribute_ids.pop(to_remove)
                # recursively call the algorithm
                child.next = self.id3_recv(child_x_ids, attribute_ids, child.next)
        return node

    # prints tree
    def printTree(self):
        # use dfs to check through tree, use recursion to print
        def dfs(node, depth):
            print('   ' * depth + node.value)
            if node.childs:
                for child in node.childs:
                    dfs(child, depth + 1)
            if node.next:
                dfs(node.next, depth + 1)
                
        dfs(self.node, 0)
    
    # gets accuracy of dataset
    def checkAccuracy(self, dataset):
        accuracyCount = 0
        size = dataset[dataset.columns[0]].size
        def explore(node, depth, dataset):
            nonlocal accuracyCount
            if node.childs:
                toex = [] # used to keep track of when to perform recursion 
                for child in node.childs:
                    if child.next:
                        if child.next.value in self.labelCategories:
                            # count amount of correctly labeled values given att and att_val                
                            for x in range(dataset[node.value].size):
                                # if the attribute value is the same as the parent node's
                                if dataset[node.value][x] == child.value:
                                    # if the category values match, then increment accuracyCount
                                    if child.next.value == dataset[dataset.columns[-1]][x]:
                                        accuracyCount += 1
                                    # dropping checked datapoint
                                    dataset.drop(x, inplace = True)                        
                            # reset indices to avoid index errors
                            dataset.reset_index(drop = True, inplace = True)
                        else:
                            toex.append((child.next, child.value)) # add next node to toex
                # iterate and explore subtrees for all elements in toex
                for i in toex:
                    subset = dataset.loc[dataset[node.value] == i[1]] 
                    subset.reset_index(drop = True, inplace = True)
                    explore(i[0], depth + 1, subset)
                    
        explore(self.node, 0, dataset)
        # calculate accuracy here
        accuracy = round(accuracyCount / size * 100, 2)
        return accuracy

In [2]:
# sample data set
# define attributes and target values
data = {
    'Outlook': ['Sunny', 'Rainy', 'Overcast'],
    'Humidity': ['Low', 'High'],
    'Wind': ['Weak', 'Strong'],
    'Play': ['Yes', 'No']
}

# create an empty dataframe
data_df = pd.DataFrame(columns=data.keys())

# randomnly create 500 instances
np.random.seed(0)
for i in range(500):
    data_df.loc[i, 'Outlook'] = str(np.random.choice(data['Outlook'], 1)[0])
    data_df.loc[i, 'Humidity'] = str(np.random.choice(data['Humidity'], 1)[0])
    data_df.loc[i, 'Wind'] = str(np.random.choice(data['Wind'], 1)[0])
    data_df.loc[i, 'Play'] = str(np.random.choice(data['Play'], 1)[0])

data_df.size
data_df.head()

,Outlook,Humidity,Wind,Play
0,Sunny,High,Strong,Yes
1,Rainy,High,Strong,Yes
2,Sunny,High,Weak,Yes
3,Sunny,Low,Weak,No
4,Overcast,High,Strong,Yes


In [3]:
# splitting data randomly into test and training sets based on desired percentage
trainPer = 0.75
training_set = data_df.sample(frac = trainPer).copy()
test_set = data_df.drop(training_set.index).reset_index(drop = True).copy()
training_set.reset_index(drop = True, inplace = True)

In [4]:
# separate target from predictors
X = np.array(data_df.drop('Play', axis=1).copy())
y = np.array(data_df['Play'].copy())
attribute_names = list(data_df.keys())[:3]

# instantiate DecisionTree
tree = DecisionTree(X, attribute_names, y)

# run algorithm id3 to build a tree
tree.id3()
tree.printTree()

Wind
   Strong
      Humidity
         Low
            Outlook
               Rainy
                  Yes
               Overcast
                  Yes
               Sunny
                  Yes
         High
            Yes
   Weak
      No


In [5]:
# check accuracy of training set of 
print(str(tree.checkAccuracy(training_set.copy())) + '%')
print(str(tree.checkAccuracy(test_set.copy())) + '%')

52.8%
53.6%


In [6]:
# create dataframe
data_df = pd.read_csv('tic-tac-toe.csv')
data_df.head()

# splitting data randomly into test and training sets based on desired percentage
trainPer = 0.75
training_set = data_df.sample(frac = trainPer).copy()
test_set = data_df.drop(training_set.index).reset_index(drop = True).copy()
training_set.reset_index(drop = True, inplace = True)

training_set.size
training_set.head()

,tl,tm,tr,ml,mm,mr,bl,bm,br,win
0,x,b,o,x,x,o,o,b,x,positive
1,x,x,x,b,o,o,b,x,o,positive
2,x,o,b,b,x,o,x,o,x,positive
3,x,x,b,x,b,o,x,o,o,positive
4,x,o,o,x,x,o,b,b,x,positive


In [7]:
test_set.size
test_set.head()

,tl,tm,tr,ml,mm,mr,bl,bm,br,win
0,x,x,x,x,o,o,o,x,o,positive
1,x,x,x,x,o,o,o,o,x,positive
2,x,x,x,x,o,o,b,b,o,positive
3,x,x,x,x,o,b,o,o,b,positive
4,x,x,x,x,b,o,b,o,o,positive


In [8]:
# separate target from predictors
X = np.array(data_df.drop('win', axis=1).copy())
y = np.array(data_df['win'].copy())
attribute_names = list(data_df.keys())[:9]

# instantiate DecisionTree
tree = DecisionTree(X, attribute_names, y)

# run algorithm id3 to build a tree
tree.id3()
tree.printTree()

mm
   o
      tl
         o
            br
               o
                  negative
               b
                  negative
               x
                  tm
                     o
                        tr
                           o
                              negative
                           b
                              bm
                                 o
                                    negative
                                 x
                                    positive
                           x
                              mr
                                 o
                                    positive
                                 b
                                    ml
                                       b
                                          positive
                                       x
                                          negative
                                 x
                                    positive
           

In [9]:
# check accuracy of training set
print(str(tree.checkAccuracy(training_set.copy())) + '%')
print(str(tree.checkAccuracy(test_set.copy())) + '%')

72.98%
72.5%


C:\Users\ziban\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
# Random Forest Implementation
data_df = pd.read_csv('tic-tac-toe.csv')
data_df.head()

trainPer = 0.75
train_acc = 0
test_acc = 0

for i in range(100):
    # splitting data randomly into test and training sets based on desired percentage
    training_set = data_df.sample(frac = trainPer).copy()
    test_set = data_df.drop(training_set.index).reset_index(drop = True).copy()
    training_set.reset_index(drop = True, inplace = True)
    
    # separate target from predictors
    X = np.array(data_df.drop('win', axis=1).copy())
    y = np.array(data_df['win'].copy())
    attribute_names = list(data_df.keys())[:9]

    # instantiate DecisionTree
    tree = DecisionTree(X, attribute_names, y)

    # run algorithm id3 to build a tree
    tree.id3()
    
    train_acc += tree.checkAccuracy(training_set)
    test_acc += tree.checkAccuracy(test_set)

# print average accuracy of random forest
train_acc /= 100
train_acc = round(train_acc, 2)
print(str(train_acc) + '%')
test_acc /= 100
test_acc = round(test_acc, 2)
print(str(test_acc) + '%')

72.76%
73.16%
